# Tema 9: expresiones regulares (I)
Hasta ahora, todo lo que hemos visto sobre tratamiento de strings han sido los métodos propios de ellas que vimos en el tema 3. Pero en este tema vamos a aprender a leer y escribir expresiones regulares para un tratamiento más avanzado.

## ¿Qué son las _regex_ o _expresiones regulares_?
Las regex son una serie de patrones y reglas que los gobiernan que sirven básicamente para hacer búsquedas en textos o evaluar si determinada string cumple las condiciones que nos interesen. Son tremendamente útiles para trabajar de manera avanzada con texto.

Las regex son una tecnología relativamente vieja, inventada en los 50, pero muy poderosa y simple a la vez. Esto las hace multidisciplinares, es decir, tienen todo tipo de aplicaciones y se usan en todo tipo de tareas de programación. También es importante saber que están muy extendidas, hay muchos programas que los soportan. Por supuesto, todos los lenguajes de programación, pero también programas de procesamiento de texto como Word, Writer, AntConc, TshwaneLex... y en los de texto plano en los que escribimos nuestros programas, claro. Lo "malo" es que, por esto mismo, también hay muchos "dialectos" o "sabores" (en inglés hablan de _flavors_) de las regex, pero no hay grandes diferencias entre ellos. Aquí vamos a aprenderlas en general, lo que es común a todos los dialectos, y en especial en Python.

Algunas de sus utilidades son:
- Buscar y reemplazar en texto o en código
- Validar texto de entrada
- Renombrar archivos
- Buscar archivos en la línea de comandos
- Buscar en bases de datos
- Hacer scraping
- Limpiar texto

Lo bueno de las regex es que es una de esas cosas tan populares que en internet hay miles de recursos, cursos, tutoriales, introducciones, ejercicios... Os dejo una pequeña lista con los más interesantes justo antes de empezar con el material, y con los que podéis complementar este tema si queréis.

Lo que vamos a hacer aquí es un repaso práctico, aprendiendo una de estas reglas o características cada vez, y haciendo ejercicios para interiorizarlas.

## Recursos
- [SketchEngine](https://regex.sketchengine.co.uk/) tiene ejercicios interesantes.
- [regex101](https://regex101.com) es perfecto para probar y compartir regex.
- [regexone](https://regexone.com) tiene un tutorial interactivo muy útil.

## Librería `re`

Antes de empezar con las regex, vamos a importar la librería `re` y ver los 4 métodos que vamos a usar:
- `re.search(patrón, texto)`: devuelve un objeto Match con la información sobre el primer lugar en el texto donde encuentra el patrón. Por ejemplo:

In [2]:
import re

busqueda = re.search("r", "librería")
print(busqueda)

<re.Match object; span=(3, 4), match='r'>


Fíjate en el argumento `span`: lo que está diciendo aquí Python es que la primera `r` que aparece está entre la posición 3 y la 4. Como vimos en el tema 3, «posición» aquí no quiere decir «carácter», sino «espacio entre dos caracteres», es decir, donde se sitúa normalmente la barra vertical que actúa como cursor de texto. Antes de la `l` de librería el cursor está en la posición 0, entre la `l` y la `i` está en la 1... y eso hace que la primera `r` esté entre las posiciones 3 y 4.

Podemos usar los objetos Match como booleanos que nos indican si el patrón se encuentra en el texto. Por ejemplo, si queremos definir una función que sepa asignar correctamente `ha` o `a` delante de una palabra, a un nivel muy simple y básico:

In [33]:
lista_palabras = ["trabajar", "aumentado", "presión", "acudido"]

def ha_o_a(lista):
    ''' Asigna correctamente ha o a delante de cada palabra de la lista '''
    for forma in lista:
        if re.search("ado", forma) or re.search("ido", forma):
            print("ha", forma)
        else:
            print("a", forma)

ha_o_a(lista_palabras)

a trabajar
ha aumentado
a presión
ha acudido


- `re.match(patrón, texto)`: muy parecida a la anterior, con la peculiaridad de que no busca en cualquier parte del texto, sino en el principio.

In [5]:
texto = "hola"

match = re.match("h", texto)
print(match)

<re.Match object; span=(0, 1), match='h'>


In [6]:
match = re.match("o", texto)
print(match)

None


- `re.findall(patrón, texto)`: devuelve una lista con todas las coincidencias del patrón en el texto, siempre que no se superpongan. La búsqueda se hace de izquierda a derecha y las coincidencias se devuelven en ese orden. Por ejemplo:

In [14]:
busqueda = re.findall("r", "librería")
print(busqueda)

['r', 'r']


Como ahora estamos usando solo caracteres literales (es decir, letras normales) para hacer las búsquedas, parece que no tiene mucho sentido mencionar lo del orden de izquierda a derecha. Pero las regex pueden procesar texto variable, así que las coincidencias pueden llegar a ser muy distintas. No te preocupes, lo veremos en el siguiente cuaderno.

¿Y qué quiere decir "que no se superpongan"? Mira el siguiente ejemplo:

In [15]:
busqueda = re.findall("99", "1999")
print(busqueda)

['99']


En realidad, hay dos coincidencias entre las strings `1999` y `99`: la de la centena y la decena (<code>1**99**9</code>) y la de la decena y la unidad (<code>19**99**</code>) pero, como se superponen, `.findall()` no nos devuelve más que una.

`.findall()` se puede usar, por ejemplo, para contar cuántas veces una determinada palabra o expresión aparece en un texto, dato que necesitaremos para calcular su frecuencia relativa, o la riqueza léxica del texto en cuestión, etc. Tan solo tenemos que medir la longitud de la lista que nos devuelve este método usando `len()` para obtener la cantidad de veces que aparece en el texto:

In [4]:
noticia = '''
La tensa confrontación entre el independentismo, Ciudadanos y Vox ha marcado la liturgia habitual 
de la conformación del Congreso este martes. Cada vez que un diputado de ERC o Junts per Catalunya 
prometía la Constitución en catalán, con una fórmula en la que defendía una república catalana y 
la libertad de los políticos presos, el grupo liderado por Albert Rivera y los miembros del 
partido de extrema derecha mostraban su disconformidad golpeando sus mesas, el suelo o quejándose. 

El líder de los liberales se ha puesto hasta dos veces de pie para intentar que Meritxell Batet, 
recién elegida presidenta de la Cámara baja, intercediera. "No vamos a interrumpir el proceso de 
acatamiento", le ha respondido la dirigente socialista. Estos incidentes han marcado la 
constitución de las Cortes y han anticipado lo que previsiblemente será una legislatura tensa y 
con una enorme división entre los grupos.
'''

def stats(str, texto):
    '''
    Imprime un mensaje con el total de apariciones de str en texto y su frecuencia relativa.'''
    texto = texto.lower()
    total = len(texto.split())
    count = len(re.findall(str, texto))
    print("La palabra", str, "aparece", count, "veces en este texto de", total,
          "palabras, lo que hace que tenga una frecuencia relativa de", count / total)
    
stats("el", noticia)
stats("la", noticia)
stats("catalán", noticia)
stats("constitución", noticia)

La palabra el aparece 10 veces en este texto de 145 palabras, lo que hace que tenga una frecuencia relativa de 0.06896551724137931
La palabra la aparece 13 veces en este texto de 145 palabras, lo que hace que tenga una frecuencia relativa de 0.0896551724137931
La palabra catalán aparece 1 veces en este texto de 145 palabras, lo que hace que tenga una frecuencia relativa de 0.006896551724137931
La palabra constitución aparece 2 veces en este texto de 145 palabras, lo que hace que tenga una frecuencia relativa de 0.013793103448275862


- `re.sub(patrón, reemplazo, texto)`: devuelve el texto resultante de reemplazar el patrón por el reemplazo en el texto original. Si no encuentra el patrón en el texto original, devolverá el texto original sin cambios. Por ejemplo:

In [16]:
texto = re.sub("ería", "ary", "librería")
print(texto)

library


In [3]:
texto = re.sub("eria", "ary", "librería")
print(texto)

librería


En la segunda celda, como la `i` no lleva tilde, no ha encontrado el patrón en el texto. Pero en vez de dar error, nos devuelve el texto sin cambios.

Las sustituciones se usan mucho para limpiar y normalizar datos textuales. Por ejemplo, si estamos leyendo un archivo con estructura csv (valores separados por comas) o algo que copiamos y pegamos de internet, es muy probable que no haya homogeneidad en los valores.

¿Alguna vez has buscado en la web del DLE por sufijos o prefijos y te ha salido una lista de palabras? Resulta que si las copiamos y pegamos vemos que al principio de cada una hay 4 espacios. También, que tenemos marcas de género que nos molestan para trabajar con los lemas... Para limpiar la lista podemos usar `re.sub()`:

In [9]:
# Lista (abreviada) de palabras que acaban en -iente recogidas en el DLE

lemas = '''acudiente
    cliente, ta
    cociente
    coeficiente
    decreciente
    deficiente
    dependiente, ta
    derechohabiente
    descendiente
    paciente
    pariente, ta
    parturiente
    pretendiente, ta
    prevaleciente
    simiente
    sirviente, ta
    sobrediente
    sobresaliente, ta
    sobreviviente
    superviviente
    tendiente
    teniente, ta
    terrateniente'''

lemas = lemas.split("\n")
lemas_limpios = []

for lema in lemas:
    lema = re.sub("    ", "", lema)
    lema = re.sub(", ta", "", lema)
    lemas_limpios.append(lema)
    
print(lemas_limpios)

['acudiente', 'cliente', 'cociente', 'coeficiente', 'decreciente', 'deficiente', 'dependiente', 'derechohabiente', 'descendiente', 'paciente', 'pariente', 'parturiente', 'pretendiente', 'prevaleciente', 'simiente', 'sirviente', 'sobrediente', 'sobresaliente', 'sobreviviente', 'superviviente', 'tendiente', 'teniente', 'terrateniente']


Por supuesto, hay muchos otros métodos en la librería `re` que pueden sernos útiles. Como siempre, en su [documentación](https://docs.python.org/3/library/re.html) (en inglés) tienes toda la información.

De momento esto es todo lo que debes saber para seguir; iremos viendo más funcionalidades de estos métodos según vayamos aprendiendo a usar las regex.